# সেশন ২ – মিনিমাল RAG পাইপলাইন

Foundry Local এবং sentence-transformers embeddings ব্যবহার করে একটি হালকা Retrieval-Augmented Generation পাইপলাইন তৈরি করুন।


### ব্যাখ্যা: নির্ভরতা ইনস্টলেশন
এই পাইপলাইনের জন্য ন্যূনতম প্যাকেজ ইনস্টল করা হয়:
- `foundry-local-sdk` স্থানীয় মডেল ব্যবস্থাপনার জন্য (যদি শুধুমাত্র BASE_URL পথ ব্যবহার না করা হয়)।
- `openai` সামঞ্জস্যপূর্ণ SDK কাঠামোর জন্য (কিছু ইউটিলিটি)।
- `sentence-transformers` এম্বেডিংয়ের জন্য।
- `numpy` ভেক্টর গণিতের জন্য।
পুনরায় চালানো নিরাপদ; যদি পরিবেশ ইতিমধ্যে পূর্ণ হয় তবে এড়িয়ে যেতে পারেন।


# পরিস্থিতি
এই নোটবুকটি একটি ক্ষুদ্র Retrieval-Augmented Generation (RAG) পাইপলাইন তৈরি করে যা সম্পূর্ণভাবে লোকালভাবে চলে:
- Foundry Local মডেলের সাথে সংযোগ স্থাপন করে (SDK বা BASE_URL এর মাধ্যমে স্বয়ংক্রিয়ভাবে সনাক্ত করে)।
- একটি ছোট ইন-মেমরি ডকুমেন্ট কর্পাস তৈরি করে এবং Sentence Transformers ব্যবহার করে এটিকে এম্বেড করে।
- স্বচ্ছতার জন্য সরল ভেক্টর সাদৃশ্য পুনরুদ্ধার (কোনো বাহ্যিক ইনডেক্স নয়) বাস্তবায়ন করে।
- একাধিক HTTP ফallback রুটের মাধ্যমে ভিত্তিক জেনারেশন অনুরোধ করে (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`)।
- একটি `answer()` হেল্পার প্রদান করে যা প্রাথমিক প্রচেষ্টা ব্যর্থ হলে বিকল্প মডেল ফর্ম পুনরায় চেষ্টা করে।

বৃহত্তর কর্পাস, স্থায়ী ভেক্টর স্টোর, বা মূল্যায়ন মেট্রিক্সে স্কেল করার আগে এটি একটি ডায়াগনস্টিক টেমপ্লেট হিসেবে ব্যবহার করুন (RAG মূল্যায়ন নোটবুক দেখুন)।


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### ব্যাখ্যা: মূল আমদানি
এম্বেডিং এবং স্থানীয় ইনফারেন্সের জন্য প্রয়োজনীয় মূল লাইব্রেরিগুলি লোড করা:
- SentenceTransformer ঘন ভেক্টর এম্বেডিংয়ের জন্য।
- FoundryLocalManager (ঐচ্ছিক) স্থানীয় পরিষেবা পরিচালনার জন্য।
- OpenAI ক্লায়েন্ট পরিচিত অবজেক্ট শেপের জন্য (যদিও আমরা পরে সরাসরি HTTP ব্যবহার করি)।


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### ব্যাখ্যা: খেলনা ডকুমেন্ট কর্পাস
ডোমেইন বিবৃতিগুলোর একটি ছোট ইন-মেমোরি তালিকা সংজ্ঞায়িত করে। পুনরাবৃত্তি দ্রুত এবং নিয়ন্ত্রিত রাখে যাতে ডেটা পরিচালনার পরিবর্তে পাইপলাইন প্রক্রিয়ার (পুনরুদ্ধার + ভিত্তি স্থাপন) উপর মনোযোগ থাকে।


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### ব্যাখ্যা: সংযোগ, মডেল নির্বাচন এবং এমবেডিং আরম্ভ
মজবুত সংযোগ লজিক:
1. ঐচ্ছিকভাবে স্পষ্ট `BASE_URL` (বিশুদ্ধ HTTP পথ) ব্যবহার করে, অন্যথায় FoundryLocalManager-এ ফিরে যায়।
2. `/v1/models` পরীক্ষা করে এবং সেরা মিলযুক্ত নির্দিষ্ট মডেল আইডি নির্বাচন করে (যথাযথ উপনাম > প্রামাণিক পরিবার > প্রথম উপলব্ধ)।
3. পুনরায় চেষ্টা করার লুপ বাস্তবায়ন করে, যা কনফিগারযোগ্য `FOUNDRY_CONNECT_RETRIES` এবং বিলম্ব সহ।
4. টয় কর্পাসের জন্য SentenceTransformer এমবেডিং (স্বাভাবিকীকৃত ভেক্টর) আরম্ভ করে।
5. পুনরুত্পাদনযোগ্যতার জন্য OpenAI SDK সংস্করণ ধারণ করে।
যদি সার্ভিস অনুপস্থিত থাকে, এটি ক্র্যাশ না করে সার্ভিস শুরু করার নির্দেশনা প্রিন্ট করে।


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### ব্যাখ্যা: Retrieve ফাংশন (ভেক্টর সাদৃশ্য)
`retrieve(query, k=3)` কোয়েরি এনকোড করে, কসাইন সাদৃশ্য (স্বাভাবিকীকৃত ভেক্টরের উপর ডট প্রোডাক্ট) গণনা করে এবং শীর্ষ-k ডকুমেন্ট সূচকগুলি ফেরত দেয়। এটি স্বচ্ছতার জন্য সহজ এবং ইন-মেমোরি থাকে।


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### ব্যাখ্যা: SDK-ভিত্তিক জেনারেশন এবং উত্তর সহায়ক
Foundry Local SDK এবং OpenAI-সামঞ্জস্যপূর্ণ ক্লায়েন্ট পদ্ধতি ব্যবহার করে পুনর্গঠিত, যেখানে ম্যানুয়াল কাঁচা HTTP পোস্টের পরিবর্তে নিম্নলিখিত পদ্ধতিগুলি ব্যবহৃত হয়:
- প্রধান পথ: `client.chat.completions.create` (গঠনমূলক বার্তা)।
- বিকল্প পদ্ধতি: `client.completions.create` (পুরনো প্রম্পট) এবং তারপর `client.responses.create` (সরলীকৃত প্রতিক্রিয়া API)।
- বিকল্প মডেল আইডি (RAW বনাম সরলীকৃত ALT) স্বাভাবিকীকরণ করে সামঞ্জস্যতা বাড়ানো হয়েছে।
- `answer()` শীর্ষ-k পুনরুদ্ধার করা ডকুমেন্ট থেকে একটি ভিত্তিক প্রম্পট তৈরি করে এবং ক্রমবদ্ধ প্রচেষ্টার ট্রেস রেকর্ড করে।
এটি যুক্তিকে পাঠযোগ্য রাখে, একই সাথে OpenAI-সামঞ্জস্যপূর্ণ এন্ডপয়েন্টগুলির বিবর্তনের সাথে সামঞ্জস্য রেখে মসৃণ ডিগ্রেডেশন প্রদান করে।


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### ব্যাখ্যা: ব্যাচ কোয়েরি স্মোক টেস্ট
`answer()` এর মাধ্যমে বেশ কয়েকটি প্রতিনিধিত্বমূলক ব্যবহারকারীর প্রশ্ন চালানো হয় যাচাই করার জন্য:
- পুনরুদ্ধারের সূচকগুলি যুক্তিসঙ্গত সহায়ক ডকুমেন্টের সাথে সঙ্গতিপূর্ণ।
- ফallback রাউটিং কাজ করে (route মান 'failed' নয়)।
- উত্তরগুলি ভিত্তি নির্দেশনা অনুসরণ করে (কোনও ভুল তথ্য নয়)।
শেষ ফলাফল অবজেক্টটি অ্যাড-হক পরিদর্শনের জন্য সংরক্ষণ করে।


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### ব্যাখ্যা: একক উত্তর সুবিধাজনক কল
সহজ কপি/পেস্ট পুনঃব্যবহার বা পরবর্তী রেফারেন্সিংয়ের জন্য চূড়ান্ত দ্রুত একক-প্রশ্ন কল। পূর্ববর্তী প্রস্তুতিমূলক প্রশ্নের পরে `answer()` এর ইডেমপোটেন্ট ব্যবহার প্রদর্শন করে।


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
